This notebook combines the CellProfiller extracted morphology features and the scDINO extracted morphology features into one feature space. Downstream notebooks will normalize the data and perform feature selection.

In [1]:
import pathlib

import numpy as np
import pandas as pd

In [4]:
# define data paths for import
cellprofiller_annotated_features_path = pathlib.Path(
    "../../5.process_CP_features/data/3.combined_data/profiles/combined_data.parquet"
).resolve(strict=True)

scdino_features = pathlib.Path(
    "../../6.scDINO_analysis/1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)

# set the output path
output_path = pathlib.Path("../data/CP_scDINO_features/combined_data.parquet").resolve()

# make the parent directory
output_path.parent.mkdir(parents=True, exist_ok=True)

In [15]:
pd.options.display.max_columns = None

In [16]:
# load in the data
cellprofiller_data = pd.read_parquet(cellprofiller_annotated_features_path)
scdino_data = pd.read_parquet(scdino_features)
cellprofiller_data["features_source"] = "cellprofiler"
scdino_data["features_source"] = "scDINO"

print(f"CellProfiller data shape: {cellprofiller_data.shape}")
print(f"scDINO data shape: {scdino_data.shape}")
cellprofiller_data.head()

CellProfiller data shape: (209311, 2333)
scDINO data shape: (182804, 1546)


Metadata_plate Metadata_Well  Metadata_number_of_singlecells   
0               1          D-02                             192  \
1               1          D-02                             192   
2               1          D-02                             192   
3               1          D-02                             192   
4               1          D-02                             192   

  Metadata_compound  Metadata_dose Metadata_control  Metadata_ImageNumber   
0     Staurosporine            0.0         negative                     1  \
1     Staurosporine            0.0         negative                     1   
2     Staurosporine            0.0         negative                     1   
3     Staurosporine            0.0         negative                     1   
4     Staurosporine            0.0         negative                     1   

  Metadata_FOV  Metadata_Time  Metadata_Cells_Number_Object_Number   
0         0001            0.0                                    2  \
1         0001            0.0                                   23   
2         0001            0.0                                   31   
3         0001            0.0                                   35   
4         0001            0.0                                   40   

   Metadata_Cytoplasm_Parent_Cells  Metadata_Cytoplasm_Parent_Nuclei   
0                                2                                 2  \
1                               23                                23   
2                               31                                30   
3                               35                                35   
4                               40                                38   

   Metadata_ImageNumber_1  Metadata_ImageNumber_2  Metadata_ImageNumber_3   
0                       1                       1                     1.0  \
1                       1                       1                     1.0   
2                       1                       1                     1.0   
3                       1                       1                     1.0   
4                       1                       1                     1.0   

   Metadata_Nuclei_Number_Object_Number   
0                                   2.0  \
1                                  23.0   
2                                  30.0   
3                                  35.0   
4                                  38.0   

               Metadata_Image_FileName_CL_488_1   
0  D-02_F0001_T0001_Z0001_C02_illumcorrect.tiff  \
1  D-02_F0001_T0001_Z0001_C02_illumcorrect.tiff   
2  D-02_F0001_T0001_Z0001_C02_illumcorrect.tiff   
3  D-02_F0001_T0001_Z0001_C02_illumcorrect.tiff   
4  D-02_F0001_T0001_Z0001_C02_illumcorrect.tiff   

               Metadata_Image_FileName_CL_488_2   
0  D-02_F0001_T0001_Z0001_C03_illumcorrect.tiff  \
1  D-02_F0001_T0001_Z0001_C03_illumcorrect.tiff   
2  D-02_F0001_T0001_Z0001_C03_illumcorrect.tiff   
3  D-02_F0001_T0001_Z0001_C03_illumcorrect.tiff   
4  D-02_F0001_T0001_Z0001_C03_illumcorrect.tiff   

                 Metadata_Image_FileName_CL_561   
0  D-02_F0001_T0001_Z0001_C04_illumcorrect.tiff  \
1  D-02_F0001_T0001_Z0001_C04_illumcorrect.tiff   
2  D-02_F0001_T0001_Z0001_C04_illumcorrect.tiff   
3  D-02_F0001_T0001_Z0001_C04_illumcorrect.tiff   
4  D-02_F0001_T0001_Z0001_C04_illumcorrect.tiff   

                    Metadata_Image_FileName_DNA   
0  D-02_F0001_T0001_Z0001_C01_illumcorrect.tiff  \
1  D-02_F0001_T0001_Z0001_C01_illumcorrect.tiff   
2  D-02_F0001_T0001_Z0001_C01_illumcorrect.tiff   
3  D-02_F0001_T0001_Z0001_C01_illumcorrect.tiff   
4  D-02_F0001_T0001_Z0001_C01_illumcorrect.tiff   

                    Metadata_Image_PathName_CL_488_2   
0  /gpfs/alpine1/scratch/mlippincott@xsede.org/li...  \
1  /gpfs/alpine1/scratch/mlippincott@xsede.org/li...   
2  /gpfs/alpine1/scratch/mlippincott@xsede.org/li...   
3  /gpfs/alpine1/scratch/mlippincott@xsede.org/li...   
4  /gpfs/alpine1/scratch/mlippincott@xsede.org/li... 

In [17]:
scdino_data.head()

Metadata_image_path Metadata_Well   
0  ../0.pre-process_images/data/processed_images/...          C-02  \
1  ../0.pre-process_images/data/processed_images/...          C-02   
2  ../0.pre-process_images/data/processed_images/...          C-02   
3  ../0.pre-process_images/data/processed_images/...          C-02   
4  ../0.pre-process_images/data/processed_images/...          C-02   

  Metadata_FOV Metadata_Time Metadata_ImageNumber   
0         0001          0001                    1  \
1         0001          0001                    1   
2         0001          0001                    1   
3         0001          0001                    1   
4         0001          0001                    1   

  Metadata_Nuclei_Number_Object_Number Metadata_compound  Metadata_dose   
0                                 cell     Staurosporine            0.0  \
1                                 cell     Staurosporine            0.0   
2                                 cell     Staurosporine            0.0   
3                                 cell     Staurosporine            0.0   
4                                 cell     Staurosporine            0.0   

  Metadata_control  channel488-1_cls_feature_0  channel488-1_cls_feature_1   
0         negative                   -0.012062                   -0.043927  \
1         negative                   -0.014538                   -0.018479   
2         negative                   -0.010044                   -0.093711   
3         negative                    0.021043                   -0.030516   
4         negative                   -0.009109                   -0.043035   

   channel488-1_cls_feature_10  channel488-1_cls_feature_100   
0                     0.040292                      0.021701  \
1                     0.047950                     -0.008538   
2                     0.018404                      0.011412   
3                     0.051163                     -0.007309   
4                    -0.013794                     -0.009745   

   channel488-1_cls_feature_101  channel488-1_cls_feature_102   
0                      0.052456                      0.056071  \
1                      0.088175                      0.035160   
2                      0.014773                      0.029010   
3                      0.050966                     -0.003750   
4                      0.060354                      0.006337   

   channel488-1_cls_feature_103  channel488-1_cls_feature_104   
0                     -0.175999                      0.049407  \
1                     -0.197851                      0.045439   
2                     -0.119090                      0.071326   
3                     -0.137231                      0.062148   
4                     -0.211874                      0.024206   

   channel488-1_cls_feature_105  channel488-1_cls_feature_106   
0                      0.003097                      0.023554  \
1                     -0.006583                      0.040301   
2                      0.006202                      0.052619   
3                      0.025259                      0.045957   
4                      0.023032                      0.035577   

   channel488-1_cls_feature_107  channel488-1_cls_feature_108   
0                      0.064245                     -0.020793  \
1                      0.069246                     -0.044414   
2                      0.032237                     -0.047702   
3                      0.063700                     -0.039527   
4                      0.061569                     -0.026995   

   channel488-1_cls_feature_109  channel488-1_cls_feature_11   
0                      0.051653                    -0.003998  \
1                      0.039493                     0.002900   
2                      0.055657                    -0.049117   
3                      0.050884                     0.014300   
4                      0.057926                    -0.030276   

   channel488-1_cls_feature_110  channe

In [ ]:
scdino_data["Metadata_image_path"][0].split("sc_crops/")[-1].split("/")[0]
well

'C-02_F0001_time_0.0_image_number_1_cell_number_100.0'

In [ ]:
merge_columns = [
    "Metadata_Well",
    "Metadata_FOV",
    "Metadata_Time",
    "Metadata_ImageNumber",
    "Metadata_Nuclei_Number_Object_Number",
]

In [ ]:
pixel_cutt_off = 5
left_on = ["Metadata_Time"]
right_on = ["Metadata_t"]
merged_df_list = []  # list to store the merged dataframes
total_CP_cells = 0  # total number of cells in the left dataframe
total_annotated_cells = 0  # total number of cells that were annotated
distances = []  # list to store the distances between the coordinates


# In[ ]:


for time in cellprofiller_data["Metadata_Time"].unique():
    df_left = cellprofiller_data.copy().loc[cellprofiller_data["Metadata_Time"] == time]
    df_right = scdino_data.copy().loc[scdino_data["Metadata_t"] == time]

    total_CP_cells += df_left.shape[0]
    # loop through the rows in the subset_annotated_df and find the closest coordinate set in the location metadata
    for index1, row1 in df_left.iterrows():
        dist = np.inf
        for index2, row2 in df_right.iterrows():
            coord1 = row1[coordinate_column_left]
            coord2 = row2[coordinate_column_right]
            try:
                temp_dist = np.linalg.norm(np.array(coord1) - np.array(coord2))
            except:
                temp_dist = np.inf
            if temp_dist <= dist:
                dist = temp_dist
                coord2_index = index2

            # set cut off of 5,5 pixel in the euclidean distance
            euclidean_cut_off = np.linalg.norm(
                np.array([0, 0]) - np.array([pixel_cutt_off, pixel_cutt_off])
            )

        if dist < euclidean_cut_off:
            temp_merged_df = pd.merge(
                df_left.loc[[index1]],
                df_right.loc[[coord2_index]],
                how="inner",
                left_on=left_on,
                right_on=right_on,
            )
            distances.append(dist)
            total_annotated_cells += temp_merged_df.shape[0]
            merged_df_list.append(temp_merged_df)
if len(merged_df_list) == 0:
    merged_df_list.append(pd.DataFrame())
merged_df = pd.concat(merged_df_list)
merged_df["Metadata_distance"] = distances

# replace Metadata string in column names with Metadata (Non Morphology Features)
merged_df.columns = [
    x.replace("Metadata_", "Metadata_") if "Metadata_" in x else x
    for x in merged_df.columns
]

print(f"Annotated cells: {total_annotated_cells} out of {total_CP_cells}")
print(f"Percentage of annotated cells: {total_annotated_cells/total_CP_cells*100}%")
print(merged_df.shape)
merged_df.to_parquet(profiles_output_dir / f"{well_fov}_annotated_tracks.parquet")
merged_df.head()